In [ ]:
import pandas as pd
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import re
import time

In [25]:
chrome_options = Options()
force_headless = False
if GLOBAL_CONFIG['simulator']['headless'] or force_headless:
    print("Running in headless mode")
    chrome_options.add_argument("--headless")
chrome_options.add_argument("--log-level=3")
chrome_options.add_argument("--lang=en-GB")
chrome_options.add_argument("--disable-web-security")
chrome_options.add_argument(f"user-data-dir={GLOBAL_CONFIG['simulator']['user_data_dir']}")
chrome_options.binary_location = GLOBAL_CONFIG['simulator']['binary_location']

driver = webdriver.Chrome(options=chrome_options)

In [ ]:
directory = "log/session_scripts"
output_directory = "log/session_scripts/evaluated"
# Evaluate specific files in the directory
task = input("Input task to run the script: ")
data = pd.read_excel(os.path.join(directory, f'{task}_evaluation.xlsx'))
results = []
reasons = []
revised_scripts = []

for idx, row in data.iterrows():
    script = row['Test Script']
    print(f"\nRunning test {idx+1} for {task}:")
    print(f"Task: {row['Task']}")

    modified_lines = []
    insert_sleep_after = [
        r'driver\.get\(',
        r'\.click\(',
        r'\.send_keys\(',
        r'\.clear\(',
        r'wait\.until\('
    ]
    sleep_line = 'time.sleep(3)'

    # Ensure import time exists
    if 'import time' not in script:
        modified_lines.append('import time')

    for line in script.splitlines():
        modified_lines.append(line)
        for pattern in insert_sleep_after:
            if re.search(pattern, line.strip()):
                indent = ' ' * (len(line) - len(line.lstrip()))
                modified_lines.append(f'{indent}{sleep_line}')
                break  # Don't insert multiple sleeps per line

    modified_script = '\n'.join(modified_lines)
    print("Modified script with sleep statements:")
    print(modified_script)

    try:
        exec_globals = {'driver': driver}
        exec(modified_script, exec_globals)
        print("Test script executed.")
    except Exception as e:
        print(f"Error during test script execution: {e}")
    result = input("Enter the test result (e.g., Valid/Invalid): ")
    if result == "Fail":
        reason = input("Enter invalid reason: ")
    else:
        reason = None

    results.append(result)
    reasons.append(reason)
    revised_scripts.append(modified_script)

data['Revised Test Script'] = revised_scripts
data['Test Result'] = results    
data['Test Fail Reason'] = reasons
data.to_excel(os.path.join(output_directory, f'{task}_evaluation.xlsx'), index=False)
